# Models training helper functions - note these are Helper Classes with helper methods, this file should contain *no* stand alone executable code.

###This file contains 4 seperate heplper classes
    -class generic_parsing_helpers:
    -class AIVM_helper: (including github read/write and parsing methods
    -class roc_callback(Callback)
    -class model_helpers:


### This file contains the following helper files:

#### read_glove_embeddings,  vocabulary loading
def read_glove_embeddings():


#### get_stop_words, returns an immutable frozenset set of stopwords from a file.
def get_stop_words(stop_file_path):


#### clean_line, cleaning function which calls a stemmer, cleans the text and returns a string
def clean_line(X):


#### def create_tokenizer, create a tokenizer given a data frame with a column "text"
def create_tokenizer(df, max_words=MAX_NB_WORDS):


#### roc_callback, calculate AUC for validation when training the model
##As explained here: https://stackoverflow.com/questions/41032551/how-to-compute-receiving-##operating-characteristic-roc-and-auc-in-keras
class roc_callback(Callback):


 
AF : this file should be retained as a helper file so it should not really make any calls
but rather just have methods dafined. 

it's calling: 
embeddings_index = read_glove_embeddings() 
stopwords=get_stop_words("Data/Iterative-models-building/Training data/resources/stopwords.txt")


AF:
Questions.
1. Where does the stopwords list come from? is there a reason not to use the NLTK onne instead of storing locally?


In [45]:
# Required Python utilities
import numpy as np
import pandas as pd
from collections import Counter
import re
from langdetect import detect
from bs4 import BeautifulSoup
from markdown import markdown
from lxml import etree
import random
import tqdm
import itertools 
import pickle
import os
import base64
import io
from io import BytesIO
import urllib 
from collections import Counter
from langdetect import detect
from bs4 import BeautifulSoup
from lxml import etree
import time
from PIL import Image as PIL_Img
from datetime import datetime
import json
import gc
import requests as req
import markdown



from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

## Deep Learning imports for the classifiers
os.environ['KERAS_BACKEND']='theano'

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, Concatenate
from keras.models import Model

##Â ML required imports (for clustering)
from sklearn import metrics
from sklearn.decomposition import PCA, LatentDirichletAllocation
from sklearn.preprocessing import scale, StandardScaler
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, DBSCAN
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB, GaussianNB

# Topic modeling imports
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

##Â NLP related imports
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer


# visualization imports
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg



import ipywidgets as widgets
from ipywidgets import interact, interact_manual


from github import Github, GithubException, InputGitTreeElement
from IPython.display import display, clear_output, HTML, Image



print ("Necessary libraries imported.")

Necessary libraries imported.


# Defining a default model_helpers for the project.

In [3]:
def default_model_helpers_for_project ():
    
    #Classificaiton NETWORKs Configuration parameters
    SEQUENCE_LENGTH = 32
    MAX_NB_WORDS = 10000
    EMBEDDING_DIM = 100 # 100, 200 or 300
    VALIDATION_SPLIT = 0.2
    TFIDF_MAX_FEATURES=10000
    NUM_EPOCHS = 20
    
    #from "Models training helpers.ipynb". model_helpers class is defined further below. 
    def_model_helpers = model_helpers (seq_len=SEQUENCE_LENGTH, 
                                   max_num_wrds=MAX_NB_WORDS, 
                                   embedding_dim=EMBEDDING_DIM,
                                   validation_splt =VALIDATION_SPLIT,
                                   tfidf_max = TFIDF_MAX_FEATURES,
                                   num_epochs = NUM_EPOCHS)
  

    HTML_list = []
    HTML = ''      
    HTML_list.append ('You will find instantiate_model_helpers_for_project at the very TOP of All_helper_classes.ipynb, If you do not want to change the default config, used across all files you can instantiate your own model_helpers() instead.')
    HTML_list.append('<br><b>SEQUENCE_LENGTH:</b> '+ str(SEQUENCE_LENGTH))
    HTML_list.append('<br><b>MAX_NB_WORDS:</b> '+ str(MAX_NB_WORDS))
    HTML_list.append('<br><b>EMBEDDING_DIM:</b> '+ str(EMBEDDING_DIM))
    HTML_list.append('<br><b>VALIDATION_SPLIT:</b> ' + str(VALIDATION_SPLIT))
    HTML_list.append('<br><b>TFIDF_MAX_FEATURES:</b> ' + str(TFIDF_MAX_FEATURES))
    HTML_list.append('<br><b>NUM_EPOCHS '  + str(NUM_EPOCHS))
    for line in HTML_list:
            HTML = HTML + line 
            
    return def_model_helpers, HTML

# Defining a default list of paths for the project.

In [55]:
# If you add any paths to this make sure to also add to the display command 
# so the calling notebook can see all available paths
class project_paths:
    def __init__(self):
        #self.something = kwargs['something']
        
        self.GIT_OWNER = 'aideenf'
        self.GIT_REPO = 'AIVC'
        self.GIT_OWNER_REPO = self.GIT_OWNER + '/' + self.GIT_REPO
        self.BASE_DIR_LOCAL = './Data/Iterative-models-building'
        self.GLOVE_DIR_LOCAL = 'Data/Iterative-models-building/Training data/'

        self.GATHERED_DATA_CONV_DIR_LOCAL = 'Data/Iterative-models-building/Gathered_data/Conventions/'
        self.GATHERED_DATA_CONV_DIR_GIT = 'cp_wssc/Data/Iterative-models-building/Gathered_data/Conventions/'
        self.GITHUB_GATHERED_URL_PATH = 'https://raw.githubusercontent.com/aideenf/AIVC/master/cp_wssc/Data/Iterative-models-building/Gathered_data/Conventions/'

        self.MODELS_DIR_LOCAL = "./Data/Iterative-models-building/Models/"

        self.STOP_WORDS_DIR_FILE_LOCAL = 'Data/Iterative-models-building/Training data/resources/stopwords.txt'
        self.STOP_WORDS_URL_GIT = 'https://raw.githubusercontent.com/aideenf/AIVC/master/cp_wssc/Data/Iterative-models-building/Training data/resources/stopwords.txt'

        self.TRAINING_DATA_DIR_FILE_LOCAL = 'Data/Iterative-models-building/Training data/Conventions/training_data_original.tsv'
        self.TRAINING_DATA_DIR_FILE_GIT = 'https://raw.githubusercontent.com/aideenf/AIVC/master/cp_wssc/Data/Iterative-models-building/Training%20data/Conventions/training_data_original.tsv'

    
        
        self.TRAINING_DATA_ALL_DIR_FILE_GIT = 'https://raw.githubusercontent.com/aideenf/AIVC/master/cp_wssc/Data/Iterative-models-building/Training%20data/Conventions/training_ALL.tsv'
        self.TRAINING_DATA_ALL_DIR_FILE_LOCAL = 'Data/Iterative-models-building/Training data/Conventions/training_ALL.tsv'
    
        
        self.TRAINING_DATA_ALL_MULTI_DIR_FILE_GIT = 'https://raw.githubusercontent.com/aideenf/AIVC/master/cp_wssc/Data/Iterative-models-building/Training%20data/Conventions/training_ALL_Multi_format.tsv'
        self.TRAINING_DATA_ALL_MULTI_DIR_FILE_LOCAL = 'Data/Iterative-models-building/Training data/Conventions/training_ALL_Multi_format.tsv'
        
        
        self.TRAINING_DATA_DIR_LOCAL = 'Data/Iterative-models-building/Training data/Conventions/'
        self.TRAINING_DATA_DIR_GIT = 'cp_wssc/Data/Iterative-models-building/Training data/Conventions/'

        self.AUDITED_DATA_DIR_LOCAL  = './Data/Iterative-models-building/Classification results/Conventions/Audited/'
        self.AUDITED_DATA_DIR_GIT =  'cp_wssc/Data/Iterative-models-building/Classification results/Conventions/Audited/'
        self.GITHUB_AUDIT_URL_PATH = 'https://raw.githubusercontent.com/aideenf/AIVC/master/cp_wssc/Data/Iterative-models-building/Classification%20results/Conventions/Audited/'
        self.GITHUB_AGGREGATED_AUDIT_URL_FILE = 'https://raw.githubusercontent.com/aideenf/AIVC/master/cp_wssc/Data/Iterative-models-building/Classification%20results/Conventions/Audited/audited_ALL.tsv'

        self.CLASSIFIED_DATA_DIR_LOCAL  = './Data/Iterative-models-building/Classification results/Conventions/'
        self.CLASSIFIED_DATA_DIR_GIT =  'cp_wssc/Data/Iterative-models-building/Classification results/Conventions/'
        

        self.CONV_MODEL_PCKL_URL = 'https://github.com/aideenf/AIVC/blob/master/cp_wssc/Data/Iterative-models-building/Models/conv_models_items.pickle?raw=true'
        self.SIMPLER_MODEL_PCKL_URL = 'https://github.com/aideenf/AIVC/blob/master/cp_wssc/Data/Iterative-models-building/Models/Simpler_ML_models.pickle?raw=true'

        if not os.path.exists(self.GLOVE_DIR_LOCAL):
            os.makedirs(self.GLOVE_DIR_LOCAL)

        if not os.path.exists(self.GATHERED_DATA_CONV_DIR_LOCAL):
            os.makedirs(self.GATHERED_DATA_CONV_DIR_LOCAL)

        if not os.path.exists(self.MODELS_DIR_LOCAL):
            os.makedirs(self.MODELS_DIR_LOCAL)
    
        if not os.path.exists(self.AUDITED_DATA_DIR_LOCAL):
            os.makedirs(self.AUDITED_DATA_DIR_LOCAL)
            
        if not os.path.exists(self.TRAINING_DATA_DIR_FILE_LOCAL):
            os.makedirs(self.TRAINING_DATA_DIR_FILE_LOCAL)
            
    def get_paths_data (self):
        HTML_list = []
        HTML = ''
        HTML_list.append("<b>*****PROJECT PATHS*******</b>")    
        HTML_list.append('<br><b>paths.BASE_DIR_LOCAL: </b>' + self.BASE_DIR_LOCAL)
        HTML_list.append('<br><b>paths.GLOVE_DIR_LOCAL: </b>' + self.GLOVE_DIR_LOCAL)
        HTML_list.append('<br><b>paths.GATHERED_DATA_CONV_DIR_LOCAL: </b>' + self.GATHERED_DATA_CONV_DIR_LOCAL) 
        HTML_list.append('<br><b>paths.GATHERED_DATA_CONV_DIR_GIT: </b>' + self.GATHERED_DATA_CONV_DIR_GIT)
        HTML_list.append('<br><b>paths.GITHUB_GATHERED_URL_PATH: </b>' + self.GITHUB_GATHERED_URL_PATH) 
        HTML_list.append('<br><b>paths.MODELS_DIR_LOCAL: </b>' + self.MODELS_DIR_LOCAL)
        HTML_list.append('<br><b>paths.STOP_WORDS_DIR_FILE_LOCAL: </b>' + self.STOP_WORDS_DIR_FILE_LOCAL) 
        HTML_list.append('<br><b>paths.STOP_WORDS_URL_GIT: </b>' + self.STOP_WORDS_URL_GIT) 
        HTML_list.append('<br><b>paths.TRAINING_DATA_DIR_FILE_LOCAL: </b>' + self.TRAINING_DATA_DIR_FILE_LOCAL) 
        HTML_list.append('<br><b>paths.TRAINING_DATA_DIR_FILE_GIT: </b>' + self.TRAINING_DATA_DIR_FILE_GIT)
        HTML_list.append('<br><b>paths.TRAINING_DATA_ALL_DIR_FILE_GIT: </b>' +self.TRAINING_DATA_ALL_DIR_FILE_GIT)
        HTML_list.append('<br><b>paths.TRAINING_DATA_ALL_DIR_FILE_LOCAL: </b>' +self.TRAINING_DATA_ALL_DIR_FILE_LOCAL)
        HTML_list.append('<br><b>paths.TRAINING_DATA_ALL_MULTI_DIR_FILE_GIT: </b>' +self.TRAINING_DATA_ALL_MULTI_DIR_FILE_GIT)
        HTML_list.append('<br><b>paths.TRAINING_DATA_ALL_MULTI_DIR_FILE_LOCAL: </b>' +self.TRAINING_DATA_ALL_MULTI_DIR_FILE_LOCAL)
        HTML_list.append('<br><b>paths.TRAINING_DATA_DIR_LOCAL: </b>' + self.TRAINING_DATA_DIR_LOCAL) 
        HTML_list.append('<br><b>paths.TRAINING_DATA_DIR_GIT: </b>' + self.TRAINING_DATA_DIR_GIT)
        HTML_list.append('<br><b>paths.AUDITED_DATA_DIR_LOCAL: </b>' + self.AUDITED_DATA_DIR_LOCAL)
        HTML_list.append('<br><b>paths.AUDITED_DATA_DIR_GIT: </b>' + self.AUDITED_DATA_DIR_GIT)
        HTML_list.append('<br><b>paths.GITHUB_AUDIT_URL_PATH: </b>' + self.GITHUB_AUDIT_URL_PATH )
        HTML_list.append('<br><b>paths.GITHUB_AGGREGATED_AUDIT_URL_FILE: </b>' + self.GITHUB_AGGREGATED_AUDIT_URL_FILE) 
        HTML_list.append('<br><b>paths.CONV_MODEL_PCKL_URL: </b>' + self.CONV_MODEL_PCKL_URL )
        HTML_list.append('<br><b>paths.SIMPLER_MODEL_PCKL_URL: </b>' + self.SIMPLER_MODEL_PCKL_URL)
        HTML_list.append('<br><b>paths.CLASSIFIED_DATA_DIR_LOCAL: </b>' + self.CLASSIFIED_DATA_DIR_LOCAL )
        HTML_list.append('<br><b>paths.CLASSIFIED_DATA_DIR_GIT: </b>' + self.CLASSIFIED_DATA_DIR_GIT)
        for line in HTML_list:
            HTML = HTML + line
            
        return HTML
    
    def get_repo_data (self):
        HTML_list = []
        HTML = ''
        HTML_list.append ('<b>*****GIT PROJECT*******</b>') 
        HTML_list.append('<br><b>paths.GIT_OWNER:</b> ' + self.GIT_OWNER)
        HTML_list.append('<br><b>paths.GIT_REPO:</b> ' + self.GIT_REPO)
        HTML_list.append('<br><b>paths.GIT_OWNER_REPO:</b> ' + self.GIT_OWNER_REPO)
        for line in HTML_list:
            HTML = HTML + line
            
        return HTML

# Generic parsing helper methods in class generic_parsing_helpers

In [ ]:
## these are inputs to the helper functions on contextual basis and should belong to a seperate file. 

class generic_parsing_helpers:
    
    def __init__(self, **kwargs ):
        #self.something = kwargs['something']
        print ("Initialised generic_parsing_helpers class and methods")
        
        
    ## Vocabulary loading
    def read_glove_embeddings(self, glove_dir, embedding_dim):
        ## Reading GLOVE (precalculated word embeddings)
        embeddings_index = {}
        f = open(os.path.join(glove_dir, 'glove.6B.{}d.txt'.format(embedding_dim)))
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        f.close()
    
        return embeddings_index
   

    ## Stop words list 
    def get_stop_words(self, stop_file_path):
        """load stop words """
    
        with open(stop_file_path, 'r', encoding="utf-8") as f:
            stopwords = f.readlines()
            stop_set = set(m.strip() for m in stopwords)
            return frozenset(stop_set)
        #load a set of stop words
    
    
    ## Text cleaning functions
    def clean_line(self, X):
    
        stemmer = WordNetLemmatizer()

        # Remove all the special characters
        line = re.sub(r'\W', ' ', str(X))

        # remove all single characters
        line = re.sub(r'\s+[a-zA-Z]\s+', ' ', line)

        # Remove single characters from the start
        line = re.sub(r'\^[a-zA-Z]\s+', ' ', line) 

        # Substituting multiple spaces with single space
        line = re.sub(r'\s+', ' ', line, flags=re.I)

        # Removing prefixed 'b'
        line = re.sub(r'^b\s+', '', line)

        # Converting to Lowercase
        line = line.lower()

        # Lemmatization
        line = line.split()

        document2 = [stemmer.lemmatize(word) for word in line]
        line = ' '.join(document2)

        return line


    def clean_str(self, text):
        documents = []
        ret = documents
        if type(text == "list"):
            #print("list: ", text)
            for X in text:
                documents.append(clean_line(X))
    
            ret =""
            if(len(documents)>1):

                for d in documents:
                    ret+=d+"\n"
            else:
                ret = documents[0]
        
        elif type(text)=="str":
            p#rint("str")
            ret = clear_line(text)
        return ret
    
    
    def remove_line_breaks(self, text_str):
        text_str = text_str.replace("\\n", " ")
        return text_str
 

    #this seems to remove line breaks from a column of data frame
    # renamed generically from def read_training_data(df_path_file, df_text_column):
    def get_clean_df_text_column(self, df, df_text_column):
        df[df_text_column] = df[df_text_column].apply(self.remove_line_breaks)
        return df


    ## Helper function to create a KERAS tokenizer apn apply to column, 'column name' of a givem pandas data frame with a column "text" and a maximum number of words to 
    ## fit_on_texts Updates internal vocabulary based on a list of texts. 
    ## This method creates the vocabulary index based on word frequency. So if you give it something like,
    ## "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 1; word_index["cat"] = 2 
    ## it is word -> index dictionary so every word gets a unique integer value. 
    ## 0 is reserved for padding. So lower integer means more frequent word (often the first few are stop words 
    ## because they appear a lot).

    def create_tokenizer(self, df, df_column_to_tokenize, max_words):
        
        texts = df[df_column_to_tokenize].values
    
        _tokenizer = Tokenizer(num_words=max_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
        _tokenizer.fit_on_texts(texts)
    
        return _tokenizer 


# Parsing and Github helpers, should check for duplications between these and generic_parsing_helpers above and consolidate.

        -markdown_to_text()
        -remove_nums_from_str(s)
        -pre_process_sentence(sent)
        -pre_process(textBlob) returns tokenized to sentences
        -remove_excluded_files(file_list)
        -remove_excluded_files_except(file_list, except_with_text)
        -rreplace(s, old, new, occurrence)
        -clean_file_name(name, replacements2=[]):
        -def save_to_github(git_user, git_password, git_repo, my_file_list, push_to_git_as):
        -def list_files_from_github_dir (owner, repo, dir_ref):
        -keep_pdf_urls_only(file_list):
        -def concat_files_from_github_dir (directory_base_url, file_list):
        -def read_single_file_from_github_dir (directory_base_url, file_name):

In [43]:
class AIVM_helper:
    
    def __init__(self):
        print("Initialised AIVM_helper class and methods")

    def image_to_byte_array(self, image: Image):
        imgByteArr = io.BytesIO()
        image.save(imgByteArr, format=image.format)
        imgByteArr = imgByteArr.getvalue()
        return imgByteArr

    def time_stamp(self):
        now = datetime.now()
        timestamp = datetime.timestamp(now)
        dt_object = datetime.fromtimestamp(timestamp)
        words = str(dt_object).split(' ')
        return words[0], words[1]

    def markdown_to_text(self, markdown_string):
        """ Converts a markdown string to plaintext """

        # md -> html -> text since BeautifulSoup can extract text cleanly
        html = markdown(markdown_string)

        # remove code snippets
        html = re.sub(r'<pre>(.*?)</pre>', ' ', html)
        html = re.sub(r'<code>(.*?)</code >', ' ', html)

        # extract text
        soup = BeautifulSoup(html, "html.parser")
        text = ''.join(soup.findAll(text=True))
        return text

    def remove_nums_from_str(self, s):
        result = ''.join([i for i in s if not i.isdigit()])
        return result

    def pre_process_sentence(self, sent):
        sent = re.sub('-', ' ', sent, flags=re.MULTILINE)  # Added by Aideen
        sent = re.sub(' +', ' ', sent, flags=re.MULTILINE)  # Added by Aideen
        sent = sent.replace(";", ", ")
        sent = re.sub(' +', ' ', sent, flags=re.MULTILINE)  # Added by Aideen
        sent = self.remove_nums_from_str(sent.replace(",", " "))
        sent = sent.replace("  ", " ")
        sent = sent.replace("#", "")# Added by Aideen
        sent = sent.replace('"', "")
        sent = sent.replace("*", "")# Added by Aideen
        sent = sent.replace(" .", ".")
        sent = sent.replace("[", "")# Added by Aideen
        sent = sent.replace("]", "")# Added by Aideen
        sent = sent.lstrip()
        sent = sent.rstrip()
        sent = sent.strip()
        return sent

    def pre_process(self, text):

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', str(text))

        # Removing splicit line change
        document = re.sub(r'\\n', '', document, flags=re.MULTILINE)

        soup = BeautifulSoup(document)

        # Remove HTML code from text
        document = soup.get_text()

        # Parse text from markdown code
        document = markdown_to_text(document)

        # Removing URLS
        document = re.sub(
            r'^https?:\/\/.*[\r\n]*', '', document, flags=re.MULTILINE)

        # Removing strings such as \\xe5 \\xe6 \\xe7 that appear a lot in the descriptions
        document = re.sub(r':?\\+x\w{2}', ' ', document, flags=re.MULTILINE)

        # Remove all the special characters except spaces, dashes, commas and dots
        document = re.sub(r"[^\s.,\-a-zA-Z0-9]", ' ', str(document))

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Substituting multiple '-' with single '-'
        document = re.sub(r'\-{2,50}', '', document, flags=re.I)

        document = re.sub('-', ' ', document,
                          flags=re.MULTILINE)  # Added by Aideen
        document = re.sub(' +', ' ', document,
                          flags=re.MULTILINE)  # Added by Aideen
        document.replace(";", ", ")
        document = re.sub(' +', ' ', document,
                          flags=re.MULTILINE)  # Added by Aideen
        # Converting to Lowercase
        document = document.lower()

        # Sentences Tokenization
        return sent_tokenize(document)

    def remove_excluded_files(self, file_list):
        cleaned_file_list = []
        for f in file_list:
            if not f.startswith('.') and not "random" in f and "gathered_" in f and not f.startswith("_"):
                cleaned_file_list.append(f)
        return cleaned_file_list

    def keep_pdf_urls_only(self, file_list):
        cleaned_file_list = []
        for f in file_list:
            if f.endswith('.pdf') and not " " in f:
                cleaned_file_list.append(f)
        return cleaned_file_list

    def remove_excluded_files_except(self, file_list, except_with_text):
        cleaned_file_list = []
        for f in file_list:
            if not f.startswith('.') and not "random" in f and except_with_text in f and not f.startswith("_"):
                cleaned_file_list.append(f)
        return cleaned_file_list

    def rreplace(self, s, old, new, occurrence):
        li = s.rsplit(old, occurrence)
        return new.join(li)


    def clean_file_name(self, name, replacements2=[]):

        replacements = [".txt", ".csv", ".tsv"]

        for r in replacements:
            name = name.replace(r, "")

        for r in replacements2:
            name = name.replace(r, "")
        return name
    

    def save_img_to_github(self, git_user, git_password, git_repo, my_file_list, push_to_git_as, commit_message):
        '''
        in order to push a file to github it must first be stored locally, then pushed
        this local location can also be local to a virtual machine. 
        takes: 
                git username, password, repo, 
                a list of files to push to git ie the full local location of file,
                a matching list of paths to push each file to in Git hub 
        '''
        user = git_user
        password = git_password
        url = git_repo
        file_list = []  # push these list of files to git
        file_names = []  # push to this location in git
        message = 'ok'
        element_list = []
        try:
            g = Github(user, password)
            try:
                repo = g.get_repo(url)
            except (IOError, OSError, GithubException) as e:
                return "error", e.message

            
            file_list = my_file_list
            file_names = push_to_git_as

            commit_message = commit_message
            master_ref = repo.get_git_ref('heads/master')
            master_sha = master_ref.object.sha
            base_tree = repo.get_git_tree(master_sha)
            

            
            for i, entry in enumerate(file_list):     
                data = open(entry, "rb").read()
                data = re.sub(b'/^(.+,)/', '' , data)
                data = base64.b64encode(data)
                blob = repo.create_git_blob(data.decode("utf-8"), "base64")
                element = InputGitTreeElement(
                    path=file_names[i], mode='100644', type='blob', sha=blob.sha)
                
                # element_list is a list of InputGitTreeElement.
                # Each one corresponds to a file.
                element_list.append(element)
            tree = repo.create_git_tree(element_list, base_tree)
            parent = repo.get_git_commit(master_sha)
            commit = repo.create_git_commit(commit_message, tree, [parent])
            print ("commit", commit)
            master_ref.edit(commit.sha)
            return commit, message
        except (IOError, OSError, GithubException) as e:
            message = "GitHub save FAILED:" + '\n' + "Are your github login credentials correct?" + \
                '\n' + "Are you a collaberator in the repo?" + '\n'
            
            return "error",  message + "Specific Error: " + str(e)
        
        
    def save_to_github(self, git_user, git_password, git_repo, my_file_list, push_to_git_as, commit_message):
        '''
        in order to push a file to github it must first be stored locally, then pushed
        this local location can also be local to a virtual machine. 
        takes: 
            git username, password, repo, 
            a list of files to push to git ie the full local location of file,
            a matching list of paths to push each file to in Git hub 
        '''
        user = git_user
        password = git_password
        url = git_repo
        file_list = []  # push these list of files to git
        file_names = []  # push to this location in git
        message = 'ok'
        element_list = []
        try:
            g = Github(user, password)
            try:
                repo = g.get_repo(url)
            except (IOError, OSError, GithubException) as e:
                return "error", e.message

            
            file_list = my_file_list
            file_names = push_to_git_as

            commit_message = commit_message
            master_ref = repo.get_git_ref('heads/master')
            master_sha = master_ref.object.sha
            base_tree = repo.get_git_tree(master_sha)
            
            for i, entry in enumerate(file_list):    
                data = base64.b64encode(open(entry, "rb").read())
                if entry.endswith('.png'):
                    data = base64.b64encode(data)
                blob = repo.create_git_blob(data.decode("utf-8"), "base64")

                element = InputGitTreeElement(
                    path=file_names[i], mode='100644', type='blob', sha=blob.sha)
                
                # element_list is a list of InputGitTreeElement.
                # Each one corresponds to a file.
                element_list.append(element)
            tree = repo.create_git_tree(element_list, base_tree)
            parent = repo.get_git_commit(master_sha)
            commit = repo.create_git_commit(commit_message, tree, [parent])
            print ("commit", commit)
            master_ref.edit(commit.sha)
            return commit, message
        except (IOError, OSError, GithubException) as e:
            message = "GitHub save FAILED:" + '\n' + "Are your github login credentials correct?" + \
                '\n' + "Are you a collaberator in the repo?" + '\n'
            
            return "error",  message + "Specific Error: " + str(e)



    def concat_files_from_github_dir(self, directory_base_url, file_list):
        df_list = []
        for file_name, i in zip(file_list, range(len(file_list))):
            urlBase = directory_base_url
            df_list.append(pd.read_csv(urlBase+file_name,
                                       sep='\t', error_bad_lines=False))
        # NOTE: "truncated": false  we should check for truncated = true to do follow on call to get all files
        df = pd.concat(df_list)
        return df

    def read_single_file_from_github_dir(self, directory_base_url, file_name):
        df_list = []
        for file_name, i in zip(file_list, range(len(file_list))):
            urlBase = directory_base_url
            df_list.append(pd.read_csv(urlBase+file_name,
                                       sep='\t', error_bad_lines=False))
        # NOTE: "truncated": false  we should check for truncated = true to do follow on call to get all files
        df = pd.concat(df_list)
        return df
    
    
    
    def list_files_from_github_dir(self, owner, repo, path):
    
        # read data files for source data directly from github.
        # to obtain the id for the folder, navigate the tree using
        # https://api.github.com/repos/{owner}/{repo}/git/trees/master
        # e.g https://api.github.com/repos/aideenf/AIVC/git/trees/master
        # once navigated each directory will be of format
        # https://api.github.com/repos/aideenf/AIVC/git/trees/{dir_ref}
        # exampe dir_ref = 048349b4dd81d95a17129e7fcd5418bdca8309b3"
        
        
        #Note A commit, or "revision", is an individual change to a file (or set of files). 
        #It's like when you save a file, except with Git, 
        #every time you save it creates a unique ID (a.k.a. the "SHA" or "hash") 
        #that allows you to keep record of what changes were made when and by who.
        # in order to list the docs in the latest version of any folder the most up to date
        #Sha should be used. 
        
        #in order to get latest must always start at the master
        #https://api.github.com/repos/aideenf/AIVC/git/trees/master
        
        # import requests as req  #we need to ensure we do not get cached response from browser.
        
        #get the dir for each step in the path
        dir_list = path.split('/')
        gathered_files = []
        headers = {
            'Cache-Control': 'no-cache',
            'Pragma': 'no-cache',
            'If-None-Match': ''
            }
        
        MASTER = 'https://api.github.com/repos/' + owner + '/' + repo + '/git/trees/master'
        NEXT = 'https://api.github.com/repos/' + owner + '/' + repo + '/git/trees/'
        resp = req.get(MASTER,  headers=headers)
        response = json.loads(resp.text)
        try:
            for value in response['tree']:
                if value['path'] == dir_list[0]:
                    NEXT_DIR = NEXT + value['sha']
            
            for d, i in zip(dir_list, range(len(dir_list)-1)):                          
                resp = req.get(NEXT_DIR,  headers=headers)
                response = json.loads(resp.text)
                for value in response['tree']:
                    if value['path'] == dir_list[i+1]:
                        NEXT_DIR = NEXT + value['sha']
                   
        
            resp = req.get(NEXT_DIR,  headers=headers)
            response = json.loads(resp.text)
            for value in response['tree']:
                gathered_files.append(value['path'])

            return gathered_files
        except: 
            print (response['message'])
            return []
        
        
    def save_plot(self, plot, save_as, local, git, to_git, git_user, git_pswd, git_owner_repo):

        fig = plot.get_figure()
        fig.savefig(local + save_as)  
    
        if to_git == True:
            my_file_list = [local + save_as]
            push_to_git_as = [git + save_as]
    
            commit, message = self.save_img_to_github(git_user, git_pswd, git_owner_repo, my_file_list, push_to_git_as, "auto push classified conventions")

            if (commit != "error"):
                print ("File to commit:", my_file_list)
                print ("Push to git as:", push_to_git_as)
                print ("Commit: ", commit)
                display(HTML("<font color='green'><b>Files Merged and Saved!!</b></font>"))
        
            
            if (commit == "error"):
                print ("File to commit: ", my_file_list)
                print ("Push to git as: ", push_to_git_as)
                display(HTML("<font color='red'><b>Warning!!</b></font>"))
                print(message)
        
        return fig
        gc.collect()

# Calss to calculate AUC for validation when training models

In [ ]:
## Calculate AUC for validation when training the model
## As explained here: https://stackoverflow.com/questions/41032551/how-to-compute-receiving-operating-characteristic-roc-and-auc-in-keras
class roc_callback(Callback):
    def __init__(self,validation_data):#training_data,validation_data):
        #self.x = training_data[0]
        #self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
       
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)

        print("Roc-AUC on validation: {}".format(str(round(roc_val,4))))
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

# Model Training Helper methods in class model_helpers

In [57]:
class model_helpers:
    
    def __init__(self, **kwargs ):
        #self.something = kwargs['something']

        
        ## Classificaiton NETWORK Configuration parameters
        self.MAX_SEQUENCE_LENGTH = kwargs['seq_len']
        self.MAX_NB_WORDS = kwargs['max_num_wrds']
        self.EMBEDDING_DIM = kwargs['embedding_dim'] ## 100, 200 or 300
        self.VALIDATION_SPLIT = kwargs['validation_splt']
        self.TFIDF_MAX_FEATURES = kwargs['tfidf_max']
        self.NUM_EPOCHS = kwargs['num_epochs']
        print ("Initialised model_helpers class and methods")

        
        
        
        ## Training a deep learning model given training, validation data
    def train_DL_model(self, x_train, y_train, x_val, y_val, tokenizer):
    
        num_epochs=self.NUM_EPOCHS
        word_index = tokenizer.word_index
    
        print('\nNumber of elements from each class in traing and validation set ')
        print(y_train.sum(axis=0))
        print(y_val.sum(axis=0))


        embedding_matrix = np.random.random((len(word_index) + 1, self.EMBEDDING_DIM))
        for word, i in word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = embedding_vector

        embedding_layer = Embedding(len(word_index) + 1,
                                    self.EMBEDDING_DIM,
                                    weights=[embedding_matrix],
                                    input_length=self.MAX_SEQUENCE_LENGTH,
                                    trainable=False)
    

        convs = []
        filter_sizes = [3,4,5]

        sequence_input = Input(shape=(self.MAX_SEQUENCE_LENGTH,), dtype='int32')
        embedded_sequences = embedding_layer(sequence_input)

        for fsz in filter_sizes:
            l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
            l_pool = MaxPooling1D(5)(l_conv)
            convs.append(l_pool)

        l_merge = Concatenate(axis=1)(convs)#

        l_cov1= Conv1D(128, 5, activation='relu')(l_merge)
        l_pool1 = MaxPooling1D(5)(l_cov1)

        l_flat = Flatten()(l_pool1)
        l_dense = Dense(128, activation='relu')(l_flat)
        preds = Dense(2, activation='softmax')(l_dense)
    
        model = Model(sequence_input, preds)
        model.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['acc'])

    
        print("model fitting - more complex convolutional neural network")
        model.summary()
        train_history = model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=num_epochs, batch_size=50, callbacks=[roc_callback(validation_data=(x_val, y_val))])

        return model,tokenizer,x_val,y_val, train_history
    
    
    
    ## Trains a DL model without using validation dataset
    def train_DL_model_not_validation(self, x_train, y_train, tokenizer):
        num_epochs=self.NUM_EPOCHS
        word_index = tokenizer.word_index
    
        print('\nNumber of elements from each class in traing and validation set ')
        print(y_train.sum(axis=0))
        print(y_val.sum(axis=0))

        embedding_matrix = np.random.random((len(word_index) + 1, self.EMBEDDING_DIM))
        for word, i in word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = embedding_vector

        embedding_layer = Embedding(len(word_index) + 1,
                                    self.EMBEDDING_DIM,
                                    weights=[embedding_matrix],
                                    input_length=self.MAX_SEQUENCE_LENGTH,
                                    trainable=False)


        convs = []
        filter_sizes = [3,4,5]

        sequence_input = Input(shape=(self.MAX_SEQUENCE_LENGTH,), dtype='int32')
        embedded_sequences = embedding_layer(sequence_input)

        for fsz in filter_sizes:
            l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
            l_pool = MaxPooling1D(5)(l_conv)
            convs.append(l_pool)

        l_merge = Concatenate(axis=1)(convs)#

        l_cov1= Conv1D(128, 5, activation='relu')(l_merge)
        l_pool1 = MaxPooling1D(5)(l_cov1)

        l_flat = Flatten()(l_pool1)
        l_dense = Dense(128, activation='relu')(l_flat)
        preds = Dense(2, activation='softmax')(l_dense)

        model = Model(sequence_input, preds)
        model.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['acc'])

        print("model fitting - more complex convolutional neural network")
        model.summary()
        train_history = model.fit(x_train, y_train,epochs=num_epochs, batch_size=50)
        return model,tokenizer,train_history

    
    
    ## Trains one DL model for each group of sentences (within each convention)
    def train_DL_models(self, df_train,
                        data_class_column="convention", 
                        data_label_column="label",
                        df_val=None,
                        tokenizer=None,
                        random_seed=None,
                        use_validation=True):
        
        num_epochs = self.NUM_EPOCHS
        _models = {}
        _tokenizers = {}
        _data_val_x = {}
        _data_val_y = {}
        _train_histories = {}
        if tokenizer is None:        
            if df_val is None:
                tokenizer = create_tokenizer(df_train, column_to_tokenize='text', max_words=self.MAX_NB_WORDS)
            else:
                tokenizer = create_tokenizer(pd.concat([df_train, df_val]), column_to_tokenize='text', max_words=self.MAX_NB_WORDS)

        word_index = tokenizer.word_index
        print('Found %s unique tokens.' % len(word_index))

    
        for convention in df_train[data_class_column].unique():
        
            print("----------------------------------------------------------------")
            print("            {}                  ".format(convention))
            print("----------------------------------------------------------------")
        
            tmp_df_train = df_train[df_train[data_class_column] == convention]
            if use_validation:
                if df_val is None:

                    texts = tmp_df_train['text'].values
                    labels = tmp_df_train[data_label_column].values

                    sequences = tokenizer.texts_to_sequences(texts)


                    data = pad_sequences(sequences, maxlen=self.MAX_SEQUENCE_LENGTH)

                    labels = to_categorical(np.asarray(labels))
                    print('Shape of data tensor:', data.shape)
                    print('Shape of label tensor:', labels.shape)

                    indices = np.arange(data.shape[0])
                    if random_seed is not None:
                        np.random.seed(random_seed)
                    np.random.shuffle(indices)
                    data = data[indices]
                    labels = labels[indices]
                    nb_validation_samples = int(self.VALIDATION_SPLIT * data.shape[0])



                    x_train = data[:-nb_validation_samples]
                    y_train = labels[:-nb_validation_samples]
                    x_val = data[-nb_validation_samples:]
                    y_val = labels[-nb_validation_samples:]

                    _model, _tokenizer, _x_val, _y_val, _train_h = self.train_DL_model(x_train, y_train, x_val, y_val, tokenizer)
                else:

                    tmp_df_val = df_val[df_val[data_class_column] == convention]

                    train_texts = tmp_df_train['text'].values
                    train_labels = tmp_df_train[data_label_column].values

                    val_texts = tmp_df_val['text'].values
                    val_labels = tmp_df_val[data_label_column].values

                    train_sequences = tokenizer.texts_to_sequences(train_texts)
                    val_sequences = tokenizer.texts_to_sequences(val_texts)

                    x_train = pad_sequences(train_sequences, maxlen=self.MAX_SEQUENCE_LENGTH)
                    x_val = pad_sequences(val_sequences, maxlen=self.MAX_SEQUENCE_LENGTH)

                    y_train = to_categorical(np.asarray(train_labels))
                    y_val = to_categorical(np.asarray(val_labels))

                    _model, _tokenizer, _x_val, _y_val, _train_h = self.train_DL_model(x_train, y_train, x_val, y_val, tokenizer)
        
                _models[convention] = _model
                _tokenizers[convention] = _tokenizer
                _data_val_x[convention] = _x_val
                _data_val_y[convention] = _y_val
                _train_histories[convention] = _train_h
            else:
                texts = tmp_df_train['text'].values
                labels = tmp_df_train[data_label_column].values

                sequences = tokenizer.texts_to_sequences(texts)

                data = pad_sequences(sequences, maxlen=self.MAX_SEQUENCE_LENGTH)

                labels = to_categorical(np.asarray(labels))
                print('Shape of data tensor:', data.shape)
                print('Shape of label tensor:', labels.shape)

            
                x_train = data
                y_train = labels
        
                _model, _tokenizer, _train_h = self.train_DL_model_not_validation(x_train, y_train, tokenizer)

                _models[convention] = _model
                _tokenizers[convention] = _tokenizer
                _train_histories[convention] = _train_h
        

            print("\n\n\n")
        
        return (_models, _tokenizers, _data_val_x, _data_val_y, _train_histories)
    
    
    
    def store_DL_models_in_picke(self, pickle_f_name, _models, _tokenizers, _val_x, _val_y, _train_histories, path):
        _convnet_items = {}
    
        _convnet_items['model'] = _conventions_models
        _convnet_items['tokenizer'] = _conventions_tokenizers
        _convnet_items['_x_val'] = _conventions_data_val_x
        _convnet_items['_y_val'] = _conventions_data_val_y
        _convnet_items['train_history'] = _conventions_train_histories
    
        with open(os.path.join(path, ), 'wb') as f:
            # Pickle the 'data' dictionary using the highest protocol available.
            pickle.dump(_convnet_items, f, pickle.HIGHEST_PROTOCOL)
            
            
    def read_DL_models_from_pickle(self, use_git, pickle_f_name, path):
        
        if use_git == False:
            f = open( os.path.join(path, pickle_f_name), 'rb') 
            _convnet_items = pickle.load(f)
            f.close()
            
        if use_git == True:
            print ("Reading from GIT:", pickle_f_name)   
            _convnet_items = pickle.load(urllib.request.urlopen(pickle_f_name))
            
    
    
        ## Load the models from the downloaded pickle file             
        _models = _convnet_items['model'] 
        _tokenizers = _convnet_items['tokenizer'] 
        _val_x = _convnet_items['_x_val'] 
        _val_y = _convnet_items['_y_val'] 
        _train_histories = _convnet_items['train_history'] 
    
        return _models, _tokenizers, _val_x, _val_y, _train_histories
                                                                           
                                        
        
        
        
        ## Trains a TF-IDF, Naive-Bayes based classifier
    def train_new_text_pipelineNB(self, texts, labels1):
        text_clf = Pipeline([
            ('vect', CountVectorizer(max_df=0.85, stop_words=stopwords, max_features=self.TFIDF_MAX_FEATURES)),
            ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
            ('clf', MultinomialNB()),
        ])
    
        text_clf.fit(texts, labels1)  
    
        return text_clf
    
    
    def train_ML_models(self, df,
                    data_class_column="convention", 
                    data_label_column="label"):
        _models = {}  
    
        for c in df[data_class_column].unique():
            sentences = df[df[data_class_column] == c]['text'].values
            labels = df[df[data_class_column] == c][data_label_column].values
        
            m = train_new_text_pipelineNB(sentences, labels)
        
            _models[c] = m
        return _models
    
    
    ## Helpers for calculating ML model probability score of class 1 for a set of sentences
    def calculate_matches_DL(self, sentences, _models, _tokenizers):
        _models_matches = {}

        ## Getting classification confidence per model for each repo
        for model_key in _models.keys():
            
            tokenized_sentences = _tokenizers[model_key].texts_to_sequences(sentences)
        
            tokenized_sentences = pad_sequences(tokenized_sentences, maxlen=self.MAX_SEQUENCE_LENGTH)
            preds = _models[model_key].predict(tokenized_sentences)
    
            _models_matches[model_key] = preds[:,1]
        
        return _models_matches

    def calculate_matches_ML(self, _sentences, _models): 
    
        _models_matches = {}

        ## Getting classification confidence per model for each repo
        for model_key in _models.keys():
        
            preds = _models[model_key].predict_proba(_sentences)[:,1]
            _models_matches[model_key] = preds

        return _models_matches


    ## Calculates probability of sentence combining DL models and ML models predictions
    def calculate_matches_mixture(self, _sentences, _modelsML, _modelsDL, _tokenizersDL):
        _repos_matches = {}
        preds_ML = calculate_matches_ML(_sentences, _modelsML)
        preds_DL = calculate_matches_DL(_sentences, _modelsDL, _tokenizersDL)

        ## Getting classification confidence per model for each repo
        for model_key in _modelsML.keys():
            
            preds1 = preds_ML[model_key]
            preds2 = preds_DL[model_key]

            _repos_matches[model_key] = preds1+preds2
        return _repos_matches
    
    
    ################################################
    ## The files that have been produced by the audit tool are retrieved and consolodated into 
    ## a data frame, this data frame is then converted to tsv file to produce one TSV file with all audited results.
    ## This file will be called: 'audited_ALL.tsv'
    ##
    ## A copy will be created with a datestamp should historic data ever be needed: 'audited_ALL_{date}-{time}.tsv'
    ##
    ## Any other process that wishes to use this data should read 'audited_ALL.tsv' which is the latest version.'''
    ##############################
    def aggregate_audited_data_from_git (self,
                                         audited_data_dir_git,
                                         github_audit_url, 
                                         audited_data_dir_local,
                                         git_user, 
                                         git_pswd, 
                                         git_owner_repo,
                                         git_owner,
                                         git_repo) :
       
        audited_data_dir_git = audited_data_dir_git
        github_audit_url = github_audit_url
        audited_data_dir_local = audited_data_dir_local
        git_user = git_user
        git_pswd = git_pswd
        git_owner_repo = git_owner_repo
        git_owner = git_owner
        git_repo = git_repo
        aivm_helper = AIVM_helper()

        # list all files from 'Audited' folder at cp_wssc/Data/Iterative-models-building/Classification%20results/Conventions/Audited/
        # Helper file call to Returns a list of files from the github owner, repo and dir_ref
        my_file_list = aivm_helper.list_files_from_github_dir (git_owner, git_repo, audited_data_dir_git)
        ## Helper file call to Remove our standard excluded files and all files that are not "audited_training_data"
        my_file_list = aivm_helper.remove_excluded_files_except(my_file_list, "audited_training_data")
        if (len(my_file_list) > 0):
            display (my_file_list.sort())
            #Returns a dataframe of concatinated files from github, of names from file_list from the github url specified.
            audited_df = aivm_helper.concat_files_from_github_dir (github_audit_url, my_file_list)
            date, time = aivm_helper.time_stamp()
            # file_name = 'audited_ALL_' + date + '-' + time + '.gz' # If using compression
            #df.to_csv(FILE_PATH + file_name, sep='\t', compression='gzip') # If using compression

            file_name = 'audited_ALL_' + date + '-' + time + '.tsv'
            file_name_unique = 'audited_ALL.tsv'
        
            audited_df.to_csv(audited_data_dir_local + file_name, sep='\t')

            #push to github
            my_file_list = [audited_data_dir_local + file_name, audited_data_dir_local + file_name]
            push_to_git_as = [audited_data_dir_git + file_name_unique, audited_data_dir_git + file_name]

            commit, message = aivm_helper.save_to_github(git_user, git_pswd, git_owner_repo, my_file_list, push_to_git_as, "auto push aggregated audit")

            if (commit != "error"):
                print ("File to commit:", my_file_list[0])
                print ("Push to git as:", push_to_git_as[0])
                print ("Commit: ", commit)
                display(HTML("<font color='green'><b>Files Merged and Saved!!</b></font>"))
        
            
            if (commit == "error"):
                print ("File to commit: ", my_file_list[0])
                print ("Push to git as: ", push_to_git_as[0])
                display(HTML("<font color='red'><b>Warning!!</b></font>"))
                print(message)
    
            return audited_df, message
        else:
            print ("*****NO FILES RETRIEVED******")
        gc.collect()
    
    
    
    #Method to get the latest aggregate audited data and to return a pandas DB containing the aggregated content.
    def get_aggregated_gathered_data (self, use_git, 
                            gathered_data_dir_git,
                            gathered_data_dir_local,
                            github_gathered_url, 
                            git_owner,
                            git_repo) :
        use_git = use_git
        gathered_data_dir_git = gathered_data_dir_git
        gathered_data_dir_local = gathered_data_dir_local
        github_gathered_url_path = github_gathered_url
        git_owner = git_owner
        git_repo = git_repo
        
        aivm_helper = AIVM_helper()

    
        gathered_conventions_files = []
        gathered_conventions_data = {}
        gathered_all_df = pd.DataFrame(columns=['text', 'provenance'])
    
        if (use_git == True):
            gathered_conventions_files = aivm_helper.list_files_from_github_dir (git_owner, git_repo, gathered_data_dir_git)
            gathered_conventions_files = aivm_helper.remove_excluded_files(gathered_conventions_files)
                   #aivm_helper.remove_excluded_files_except(my_file_list, "audited_training_data")
            #Read conventions data file from github. 
            for f in gathered_conventions_files:
                gathered_conventions_data[aivm_helper.clean_file_name(f)] = pd.read_csv(github_gathered_url_path+f, sep='\t', error_bad_lines=False)
                 # NOTE: "truncated": false  we should check for truncated = true to do follow on call to get all files
                tmp_df = gathered_conventions_data[aivm_helper.clean_file_name(f)]
                tmp_df = tmp_df.rename(columns={"sentence": "text"})
                gathered_all_df = gathered_all_df.append(tmp_df)
                
        elif (use_git == False):
            #For each file() in gathered conventions folder
            gathered_conventions_files = [f for f in os.listdir(gathered_data_dir_local)]
            gathered_conventions_files = aivm_helper.remove_excluded_files(gathered_conventions_files) 
            for f in gathered_conventions_files:
                gathered_conventions_data[aivm_helper.clean_file_name(f)] = pd.read_csv(os.path.join(gathered_data_dir_local, f), sep='\t')
                tmp_df = gathered_conventions_data[aivm_helper.clean_file_name(f)]
                tmp_df = tmp_df.rename(columns={"sentence": "text"})
                display (tmp_df.head())
                gathered_all_df = gathered_all_df.append(tmp_df)
        
        gc.collect()
        return gathered_conventions_files, gathered_all_df
    
    
   
    def get_training_data (self, use_git, 
                            training_data_dir_file_git,
                            training_data_dir_file_local,
                            git_owner,
                            git_repo) :
        use_git = use_git
        training_data_dir_file_git = training_data_dir_file_git
        training_data_dir_file_local = training_data_dir_file_local
        git_owner = git_owner
        git_repo = git_repo
        
        parsing_helpers = generic_parsing_helpers()

    
        if (use_git == True):
            df = pd.read_csv(training_data_dir_file_git, sep='\t', error_bad_lines=False, index_col=0)
            
   
            
        elif (use_git == False):
            df = pd.read_csv(training_data_dir_file_local, sep='\t', index_col=0)
        
       
        df = parsing_helpers.get_clean_df_text_column(df, 'text')
        gc.collect()
        return df
    
    
    # Method to retrieve a pandas df created by a direct call to the audited data in github. returns the audited data 
    # in same format as the training data, so only with the new label, named 'label'
    def retrieve_aggregate_audited_data_from_git (self, path) :
        df = pd.read_csv(path, sep='\t', error_bad_lines=False)
        df = df.drop(df.columns[0], axis=1)
        df = df.drop(['old'], axis=1)
        df = df.rename(columns={'new': 'label'})
        gc.collect()
        return df
    
    
    # Method to retrieve a pandas df created by a direct call to the audited data in github. returns the audited data 
    # in same format as the training data, so only with the new label, named 'label'
    def retrieve_tsv_data_from_git (self, path) :
        df = pd.read_csv(path, sep='\t', error_bad_lines=False)
        df = df.drop(df.columns[0], axis=1)
        gc.collect()
        return df
    

    def create_label_balance (self, training_data_df, 
                              training_data_dir_local, 
                              training_data_dir_git,
                              git_user, 
                              git_pswd, 
                              git_owner_repo,
                             save_to_git):
    
        '''takes a pandas df, checks for an even balance of labels 0 and 1
        per convention, and modifies data file to ensure even distribution,
        the modified df is saved to git as a .tsv, a timestamped version is also created
        the latest one will have title  'training_ALL.tsv'
        the timestamped will have title 'training_ALL.{date}-{timme}.tsv'
        the format as currently supported by each model =  label	text	provenance	convention
        '''

        aivm_helper = AIVM_helper()
        conv_list = ['civic','domestic','green','industrial','inspired','market','project','renown']
        
        #The data format we need to return
        new_training_data_df = pd.DataFrame(columns=['label','text', 'provenance', 'convention']) 
        
        #Make sure index is ok on the input data.
        training_data_df = training_data_df.reset_index(drop=True) 
        
        #Must ensure that for each convention there is an even balance of 0 and 1 labels
        print ("The number of Class 1 v's Class 0 per convention in combined data")
        counts = training_data_df[conv_list].apply(pd.Series.value_counts)
        display (counts)
        conv_label_0 = []
        conv_label_1 = []
        # we should have the same amount of label 0 as label 1.
        # if we have too many 1,s we can fill with 0(not_convention)
        for conv in conv_list:
            balance = counts[conv][1]
            conv_one = training_data_df[(training_data_df[conv] == 1)]
            #As there is excess of label 1 we will get label 0's from the other conventions.
            conv_zero = training_data_df[(training_data_df[conv] == 0)].sample(n=balance)
            temp = pd.concat([conv_one, conv_zero], sort=False)
            #FORMAT FROM: text	provenance	civic	domestic	green	industrial	inspired	market	project	renown
            #FORMAT TO: label	text	provenance	convention
            temp = temp.rename(columns={conv: 'label'})
            temp['convention'] = conv
            temp =  temp[['label','text', 'provenance' , 'convention']]    
            new_training_data_df =  pd.concat([new_training_data_df, temp], sort=False) 


        date, time = aivm_helper.time_stamp()
        file_name = 'training_ALL_' + date + '-' + time + '.tsv'
        file_name_unique = 'training_ALL.tsv'
        file_name_two = 'training_ALL_Multi_format' + date + '-' + time + '.tsv'
        file_name_unique_two = 'training_ALL_Multi_format.tsv'
        
        new_training_data_df = new_training_data_df.reset_index(drop=True)
        new_training_data_df.to_csv(training_data_dir_local + file_name, sep='\t')
        new_training_data_df.to_csv(training_data_dir_local + file_name_unique, sep='\t')
        
        training_data_df.to_csv(training_data_dir_local + file_name_two, sep='\t')
        training_data_df.to_csv(training_data_dir_local + file_name_unique_two, sep='\t')
        
        if (save_to_git == True):
             #push to github
            my_file_list = [training_data_dir_local + file_name, 
                            training_data_dir_local + file_name,
                           training_data_dir_local + file_name_two, 
                            training_data_dir_local + file_name_two]
            
            
            push_to_git_as = [training_data_dir_git + file_name_unique, 
                              training_data_dir_git + file_name,
                              training_data_dir_git + file_name_unique_two, 
                              training_data_dir_git + file_name_two
                             ]

            commit, message = aivm_helper.save_to_github(git_user, git_pswd, git_owner_repo, my_file_list, push_to_git_as, "auto push aggregated audit")

            if (commit != "error"):
                print ("File to commit:", my_file_list[0])
                print ("Push to git as:", push_to_git_as[0])
                print ("Commit: ", commit)
                display(HTML("<font color='green'><b>Files Merged and Saved!!</b></font>"))
        
            
            if (commit == "error"):
                print ("File to commit: ", my_file_list)
                print ("Push to git as: ", push_to_git_as)
                display(HTML("<font color='red'><b>Warning!!</b></font>"))
                print(message)
        
        countsNew = pd.crosstab(new_training_data_df['label'],new_training_data_df['convention'])
        return new_training_data_df, countsNew
    
    
#OLD VERSION - DELETE LATER
#     def create_label_balance (self, new_training_data_df, 
#                               training_data_dir_local, 
#                               training_data_dir_git,
#                               git_user, 
#                               git_pswd, 
#                               git_owner_repo,
#                              save_to_git):
    
#         '''takes a pandas df, checks for an even balance of labels 0 and 1
#         per convention, and modifies data file to ensure even distribution,
#         the modified df is saved to git as a .tsv, a timestamped version is also created
#         the latest one will have title  'training_ALL.tsv'
#         the timestamped will have title 'training_ALL.{date}-{timme}.tsv'
#         '''
#         #if we don't reset the index then we remove too many records.
#         new_training_data_df = new_training_data_df.reset_index(drop=True)
    
#         aivm_helper = AIVM_helper()
    
#         #Must ensure that for each convention there is an even balance of 0 and 1 labels
#         counts = pd.crosstab(new_training_data_df['label'],new_training_data_df['convention'])
#         conv_list = new_training_data_df.convention.unique()
#         conv_label_0 = []
#         conv_label_1 = []
#         # we should have the same amount of label 0 as label 1.
#         # if we have too many 1,s we can fill with 0(not_convention)
#         for conv in conv_list:
#             print (conv)
#             print ("label 0:",counts[conv][0] )
#             print ("label 1:",counts[conv][1] )
#             conv_label_0.append(counts[conv][0])
#             conv_label_1.append(counts[conv][1])
#             excessOne = (counts[conv][1] - counts[conv][0])
#             excessZero = (counts[conv][0] - counts[conv][1])
#             if (excessZero > 0):
#                 print ("Training data size: ", new_training_data_df.shape[0])
#                 print ("Excess of label 0 by:", excessZero)
#                  #As there is excess of label 0 we can remove excessZero num of label 0's for this conventions.
#                 balance = new_training_data_df[(new_training_data_df['convention'] == conv) & (new_training_data_df['label']== 0)].sample(n=excessZero)
#                 #display(balance.head(5))
#                 print ("Balancing df to remove: ", balance.shape[0])
#                 print ("num of index to rem:", len(balance.index))
#                 new_training_data_df = new_training_data_df.drop(balance.index)
#                 print ("Training data size after removal: ", new_training_data_df.shape[0])
         
#             if (excessOne > 0):
#                 print ("Training data size: ", new_training_data_df.shape[0])
#                 print ("Excess of label 1 by: ", excessOne )
#                 #As there is excess of label 1 we can add additional label 0's from the other conventions.
#                 balance = new_training_data_df[(new_training_data_df['convention'] != conv) & (new_training_data_df['label']== 1)].sample(n=excessOne)
#                 #now we should convert thos to convention = {convention} and label = 0.
#                 balance['convention'] = conv
#                 balance['label'] = 0
#                 #display(balance.head(5))
#                 #then concatenate this to the new_training_data_df
#                 print ("Balancing df to add: ", balance.shape[0])
#                 new_training_data_df =  pd.concat([new_training_data_df, balance], sort=True) 
                
#                 print ("Training data size after addition: ", new_training_data_df.shape[0])
#             print("")
        
#         date, time = aivm_helper.time_stamp()
#         file_name = 'training_ALL_' + date + '-' + time + '.tsv'
#         file_name_unique = 'training_ALL.tsv'
        
#         new_training_data_df = new_training_data_df.reset_index(drop=True)

        
#         new_training_data_df.to_csv(training_data_dir_local + file_name, sep='\t')
#         new_training_data_df.to_csv(training_data_dir_local + file_name_unique, sep='\t')
#         if (save_to_git == True):
#             #push to github
#             my_file_list = [training_data_dir_local + file_name, training_data_dir_local + file_name]
#             push_to_git_as = [training_data_dir_git + file_name_unique, training_data_dir_git + file_name]

#             commit, message = aivm_helper.save_to_github(git_user, git_pswd, git_owner_repo, my_file_list, push_to_git_as, "auto push aggregated audit")

#             if (commit != "error"):
#                 print ("File to commit:", my_file_list[0])
#                 print ("Push to git as:", push_to_git_as[0])
#                 print ("Commit: ", commit)
#                 display(HTML("<font color='green'><b>Files Merged and Saved!!</b></font>"))
        
            
#             if (commit == "error"):
#                 print ("File to commit: ", my_file_list)
#                 print ("Push to git as: ", push_to_git_as)
#                 display(HTML("<font color='red'><b>Warning!!</b></font>"))
#                 print(message)
        
#         countsNew = pd.crosstab(new_training_data_df['label'],new_training_data_df['convention'])
#         return new_training_data_df, counts, countsNew

